In [1]:
import tensorflow as tf
import numpy as np
import os

In [6]:
def read_and_decode(tfrecords_file, batch_size):
    '''read and decode tfrecord file, generate (image, label) batches
    Args:
        tfrecords_file: the directory of tfrecord file
        batch_size: number of images in each batch
    Returns:
        image: 4D tensor - [batch_size, width, height, channel]
        label: 1D tensor - [batch_size]
    '''
    # make an input queue from the tfrecord file
    filename_queue = tf.train.string_input_producer([tfrecords_file])
    
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    img_features = tf.parse_single_example(
                                        serialized_example,
                                        features={
                                               'label': tf.FixedLenFeature([], tf.int64),
                                               'image_raw': tf.FixedLenFeature([], tf.string),
                                               })
    image = tf.decode_raw(img_features['image_raw'], tf.uint8)
    
    ##########################################################
    # you can put data augmentation here, I didn't use it
    ##########################################################
    # all the images of notMNIST are 28*28, you need to change the image size if you use other dataset.
    
    image = tf.reshape(image, [208, 208,3])
    label = tf.cast(img_features['label'], tf.float32)    
    image = tf.image.per_image_standardization(image)
    image_batch, label_batch = tf.train.batch([image, label],
                                                batch_size= batch_size,
                                                num_threads= 64, 
                                                capacity = 2000)
    return image_batch, tf.reshape(label_batch, [batch_size])

# image_batch, label_batch = read_and_decode("tfrecoard/CatVsDog.tfrecords", BATCH_SIZE)

In [7]:
def init_weight(name, shape, stddev=0.1):
    return tf.get_variable(name, 
                           shape=shape, 
                           dtype=tf.float32, 
                           initializer=tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32))

def init_biases(name, shape, constant=0.1):
    return tf.get_variable(name, 
                           shape=shape, 
                           dtype=tf.float32, 
                           initializer=tf.constant_initializer(constant))

def inference(images, batch_size, n_classes):
    '''Build the model
    Args:
        images: image batch, 4D tensor, tf.float32, [batch_size, width, height, channels]
    Returns:
        output tensor with the computed logits, float, [batch_size, n_classes]
    '''
    #conv1, shape = [kernel size, kernel size, channels, kernel numbers]
    with tf.variable_scope('conv1') as scope:
        weights = init_weight("weights", shape = [3, 3, 3, 16])
        biases = init_biases("biases", shape=[16])
        conv = tf.nn.conv2d(images, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
    
    #pool1 and norm1
    with tf.variable_scope('pooling1_lrn') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1],strides=[1,2,2,1], padding='SAME', name='pooling1')
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name=scope.name)
    
    #conv2
    with tf.variable_scope('conv2') as scope:
        weights = init_weight("weights", shape=[3, 3, 16, 16])
        biases = init_biases("biases", shape=[16])
        conv = tf.nn.conv2d(norm1, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name='conv2')
    
    #pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name=scope.name)
        pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME', name='pooling2')
    
    #local3
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = init_weight("weights", shape=[dim,128], stddev=0.005)
        biases = init_biases("biases", shape=[128])
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)    
    
    #local4
    with tf.variable_scope('local4') as scope:
        weights = init_weight("weights", shape=[128,128], stddev=0.005)
        biases = init_biases("biases", shape=[128])
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
     
    # softmax
    with tf.variable_scope('softmax_linear') as scope:
        weights = init_weight("softmax_linear", shape=[128, n_classes], stddev=0.005)
        biases = init_biases("biases", shape=[n_classes])
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
    
    return softmax_linear
 
#%%
def losses(logits, labels):
    '''Compute loss from logits and labels
    Args:
        logits: logits tensor, float, [batch_size, n_classes]
        labels: label tensor, tf.int32, [batch_size]
        
    Returns:
        loss tensor of float type
    '''
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits\
                        (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        tf.summary.scalar(scope.name+'/loss', loss)
    return loss
 
#%%
def trainning(loss, learning_rate):
    '''Training ops, the Op returned by this function is what must be passed to 
        'sess.run()' call to cause the model to train.
        
    Args:
        loss: loss tensor, from losses()
        
    Returns:
        train_op: The op for trainning
    '''
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
#%%
def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
    Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
    """
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        tf.summary.scalar(scope.name+'/accuracy', accuracy)
    return accuracy

In [19]:
N_CLASSES = 2
IMG_W = 208  # resize the image, if the input image is too large, training will be very slow.
IMG_H = 208
BATCH_SIZE = 16
CAPACITY = 2000
MAX_STEP = 1000000 # with current parameters, it is suggested to use MAX_STEP>10k
learning_rate = 0.0001 # with current parameters, it is suggested to use learning rate<0.0001
 
#%%
def run_training():
    
    # you need to change the directories to yours.
    logs_train_dir = os.path.join(os.getcwd(), '/log/')
    
    tfrecords_file ="tfrecoard/CatVsDog.tfrecords"  # tfrecode 文件
    train_batch, train_label_batch = read_and_decode(tfrecords_file, batch_size=BATCH_SIZE)
    train_batch = tf.cast(train_batch,dtype=tf.float32)   # 转化为tf 的数据格式
    train_label_batch = tf.cast(train_label_batch,dtype=tf.int64) # 转化为tf 的数据格式
    
    train_logits = inference(train_batch, BATCH_SIZE, N_CLASSES)
    train_loss = losses(train_logits, train_label_batch)        
    train_op = trainning(train_loss, learning_rate)
    train__acc = evaluation(train_logits, train_label_batch)
       
    summary_op = tf.summary.merge_all()
    sess = tf.Session()
    train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
    saver = tf.train.Saver()   #  用于保存参数
    
    sess.run(tf.global_variables_initializer())  # 初始化变量
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        for step in np.arange(MAX_STEP):
            if coord.should_stop(): break
            _, tra_loss, tra_acc = sess.run([train_op, train_loss, train__acc])
               
            if step % 50 == 0:
                print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
                summary_str = sess.run(summary_op)
                train_writer.add_summary(summary_str, step)
            
            if step % 2000 == 0 or (step + 1) == MAX_STEP:
                checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)
                
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()
        
    coord.join(threads)
    sess.close()


tf.reset_default_graph()
run_training()


Step 0, train loss = 0.69, train accuracy = 43.75%
Step 50, train loss = 0.71, train accuracy = 31.25%
Step 100, train loss = 0.67, train accuracy = 62.50%
Step 150, train loss = 0.66, train accuracy = 68.75%
Step 200, train loss = 0.70, train accuracy = 56.25%
Step 250, train loss = 0.78, train accuracy = 56.25%
Step 300, train loss = 0.60, train accuracy = 75.00%
Step 350, train loss = 0.65, train accuracy = 75.00%
Step 400, train loss = 0.69, train accuracy = 56.25%
Step 450, train loss = 0.61, train accuracy = 68.75%
Step 500, train loss = 0.62, train accuracy = 68.75%
Step 550, train loss = 0.63, train accuracy = 62.50%
Step 600, train loss = 0.65, train accuracy = 68.75%
Step 650, train loss = 0.72, train accuracy = 43.75%
Step 700, train loss = 0.45, train accuracy = 93.75%
Step 750, train loss = 0.74, train accuracy = 56.25%
Step 800, train loss = 0.43, train accuracy = 75.00%
Step 850, train loss = 0.58, train accuracy = 75.00%
Step 900, train loss = 0.42, train accuracy = 87.

Step 7650, train loss = 0.27, train accuracy = 81.25%
Step 7700, train loss = 0.32, train accuracy = 87.50%
Step 7750, train loss = 0.12, train accuracy = 93.75%
Step 7800, train loss = 0.21, train accuracy = 93.75%
Step 7850, train loss = 0.26, train accuracy = 87.50%
Step 7900, train loss = 0.26, train accuracy = 93.75%
Step 7950, train loss = 0.21, train accuracy = 87.50%
Step 8000, train loss = 0.46, train accuracy = 81.25%
Step 8050, train loss = 0.35, train accuracy = 87.50%
Step 8100, train loss = 0.56, train accuracy = 75.00%
Step 8150, train loss = 0.18, train accuracy = 87.50%
Step 8200, train loss = 0.34, train accuracy = 81.25%
Step 8250, train loss = 0.18, train accuracy = 93.75%
Step 8300, train loss = 0.19, train accuracy = 93.75%
Step 8350, train loss = 0.11, train accuracy = 100.00%
Step 8400, train loss = 0.11, train accuracy = 100.00%
Step 8450, train loss = 0.35, train accuracy = 87.50%
Step 8500, train loss = 0.21, train accuracy = 93.75%
Step 8550, train loss = 0.

Step 15100, train loss = 0.10, train accuracy = 93.75%
Step 15150, train loss = 0.02, train accuracy = 100.00%
Step 15200, train loss = 0.02, train accuracy = 100.00%
Step 15250, train loss = 0.00, train accuracy = 100.00%
Step 15300, train loss = 0.03, train accuracy = 100.00%
Step 15350, train loss = 0.20, train accuracy = 87.50%
Step 15400, train loss = 0.09, train accuracy = 93.75%
Step 15450, train loss = 0.03, train accuracy = 100.00%
Step 15500, train loss = 0.07, train accuracy = 100.00%
Step 15550, train loss = 0.02, train accuracy = 100.00%
Step 15600, train loss = 0.01, train accuracy = 100.00%
Step 15650, train loss = 0.00, train accuracy = 100.00%
Step 15700, train loss = 0.01, train accuracy = 100.00%
Step 15750, train loss = 0.01, train accuracy = 100.00%
Step 15800, train loss = 0.01, train accuracy = 100.00%
Step 15850, train loss = 0.08, train accuracy = 93.75%
Step 15900, train loss = 0.15, train accuracy = 93.75%
Step 15950, train loss = 0.02, train accuracy = 100.0

Step 22450, train loss = 0.00, train accuracy = 100.00%
Step 22500, train loss = 0.00, train accuracy = 100.00%
Step 22550, train loss = 0.00, train accuracy = 100.00%
Step 22600, train loss = 0.00, train accuracy = 100.00%
Step 22650, train loss = 0.00, train accuracy = 100.00%
Step 22700, train loss = 0.00, train accuracy = 100.00%
Step 22750, train loss = 0.02, train accuracy = 100.00%
Step 22800, train loss = 0.01, train accuracy = 100.00%
Step 22850, train loss = 0.03, train accuracy = 100.00%
Step 22900, train loss = 0.02, train accuracy = 100.00%
Step 22950, train loss = 0.00, train accuracy = 100.00%
Step 23000, train loss = 0.00, train accuracy = 100.00%
Step 23050, train loss = 0.02, train accuracy = 100.00%
Step 23100, train loss = 0.00, train accuracy = 100.00%
Step 23150, train loss = 0.03, train accuracy = 100.00%
Step 23200, train loss = 0.01, train accuracy = 100.00%
Step 23250, train loss = 0.10, train accuracy = 93.75%
Step 23300, train loss = 0.00, train accuracy = 1

Step 29800, train loss = 0.00, train accuracy = 100.00%
Step 29850, train loss = 0.00, train accuracy = 100.00%
Step 29900, train loss = 0.01, train accuracy = 100.00%
Step 29950, train loss = 0.00, train accuracy = 100.00%
Step 30000, train loss = 0.00, train accuracy = 100.00%
Step 30050, train loss = 0.08, train accuracy = 93.75%
Step 30100, train loss = 0.00, train accuracy = 100.00%
Step 30150, train loss = 0.00, train accuracy = 100.00%
Step 30200, train loss = 0.00, train accuracy = 100.00%
Step 30250, train loss = 0.00, train accuracy = 100.00%
Step 30300, train loss = 0.00, train accuracy = 100.00%
Step 30350, train loss = 0.01, train accuracy = 100.00%
Step 30400, train loss = 0.00, train accuracy = 100.00%
Step 30450, train loss = 0.00, train accuracy = 100.00%
Step 30500, train loss = 0.00, train accuracy = 100.00%
Step 30550, train loss = 0.00, train accuracy = 100.00%
Step 30600, train loss = 0.00, train accuracy = 100.00%
Step 30650, train loss = 0.01, train accuracy = 1

...

Step 376850, train loss = 0.00, train accuracy = 100.00%
Step 376900, train loss = 0.00, train accuracy = 100.00%
Step 376950, train loss = 0.00, train accuracy = 100.00%
Step 377000, train loss = 0.00, train accuracy = 100.00%
Step 377050, train loss = 0.00, train accuracy = 100.00%
Step 377100, train loss = 0.00, train accuracy = 100.00%
Step 377150, train loss = 0.00, train accuracy = 100.00%
Step 377200, train loss = 0.00, train accuracy = 100.00%
Step 377250, train loss = 0.00, train accuracy = 100.00%
Step 377300, train loss = 0.00, train accuracy = 100.00%
Step 377350, train loss = 0.00, train accuracy = 100.00%
Step 377400, train loss = 0.00, train accuracy = 100.00%
Step 377450, train loss = 0.00, train accuracy = 100.00%
Step 377500, train loss = 0.00, train accuracy = 100.00%
Step 377550, train loss = 0.00, train accuracy = 100.00%
Step 377600, train loss = 0.00, train accuracy = 100.00%
Step 377650, train loss = 0.00, train accuracy = 100.00%
Step 377700, train loss = 0.00,

Step 384050, train loss = 0.00, train accuracy = 100.00%
Step 384100, train loss = 0.00, train accuracy = 100.00%
Step 384150, train loss = 0.00, train accuracy = 100.00%
Step 384200, train loss = 0.00, train accuracy = 100.00%
Step 384250, train loss = 0.00, train accuracy = 100.00%
Step 384300, train loss = 0.00, train accuracy = 100.00%
Step 384350, train loss = 0.00, train accuracy = 100.00%
Step 384400, train loss = 0.00, train accuracy = 100.00%
Step 384450, train loss = 0.00, train accuracy = 100.00%
Step 384500, train loss = 0.00, train accuracy = 100.00%
Step 384550, train loss = 0.00, train accuracy = 100.00%
Step 384600, train loss = 0.00, train accuracy = 100.00%
Step 384650, train loss = 0.00, train accuracy = 100.00%
Step 384700, train loss = 0.00, train accuracy = 100.00%
Step 384750, train loss = 0.00, train accuracy = 100.00%
Step 384800, train loss = 0.00, train accuracy = 100.00%
Step 384850, train loss = 0.00, train accuracy = 100.00%
Step 384900, train loss = 0.00,

Step 391250, train loss = 0.00, train accuracy = 100.00%
Step 391300, train loss = 0.00, train accuracy = 100.00%
Step 391350, train loss = 0.00, train accuracy = 100.00%
Step 391400, train loss = 0.00, train accuracy = 100.00%
Step 391450, train loss = 0.00, train accuracy = 100.00%
Step 391500, train loss = 0.00, train accuracy = 100.00%
Step 391550, train loss = 0.00, train accuracy = 100.00%
Step 391600, train loss = 0.00, train accuracy = 100.00%
Step 391650, train loss = 0.00, train accuracy = 100.00%
Step 391700, train loss = 0.00, train accuracy = 100.00%
Step 391750, train loss = 0.00, train accuracy = 100.00%
Step 391800, train loss = 0.00, train accuracy = 100.00%
Step 391850, train loss = 0.00, train accuracy = 100.00%
Step 391900, train loss = 0.07, train accuracy = 93.75%
Step 391950, train loss = 0.00, train accuracy = 100.00%
Step 392000, train loss = 0.00, train accuracy = 100.00%
Step 392050, train loss = 0.00, train accuracy = 100.00%
Step 392100, train loss = 0.00, 

Step 398450, train loss = 0.00, train accuracy = 100.00%
Step 398500, train loss = 0.00, train accuracy = 100.00%
Step 398550, train loss = 0.00, train accuracy = 100.00%
Step 398600, train loss = 0.00, train accuracy = 100.00%
Step 398650, train loss = 0.00, train accuracy = 100.00%
Step 398700, train loss = 0.00, train accuracy = 100.00%
Step 398750, train loss = 0.00, train accuracy = 100.00%
Step 398800, train loss = 0.00, train accuracy = 100.00%
Step 398850, train loss = 0.00, train accuracy = 100.00%
Step 398900, train loss = 0.00, train accuracy = 100.00%
Step 398950, train loss = 0.00, train accuracy = 100.00%
Step 399000, train loss = 0.00, train accuracy = 100.00%
Step 399050, train loss = 0.00, train accuracy = 100.00%
Step 399100, train loss = 0.00, train accuracy = 100.00%
Step 399150, train loss = 0.00, train accuracy = 100.00%
Step 399200, train loss = 0.00, train accuracy = 100.00%
Step 399250, train loss = 0.00, train accuracy = 100.00%
Step 399300, train loss = 0.00,

KeyboardInterrupt: 